In [1]:
from html_document_loader import HtmlDocumentLoader
from github_blogpost_preprocessor import GithubBlogpostPreprocessor
from simple_character_text_splitter import SimpleCharacterTextSplitter
from weaviate_vector_db import WeaviateVectorDb 
from gpt35_generator import Gpt35Generator

from IPython.display import Markdown
import dotenv
import bs4
import pprint

def display_md(content):
  display(Markdown(content))

### Loader

In [2]:
uri = "https://lilianweng.github.io/posts/2023-06-23-agent/"
cache_path = "./loader_cache"
loader = HtmlDocumentLoader(uri, cache_path)
document_content = loader.load()
display(document_content[0:1500])

'<!DOCTYPE html>\n<html lang="en" dir="auto">\n\n<head><meta charset="utf-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n<meta name="robots" content="index, follow">\n<title>LLM Powered Autonomous Agents | Lil&#39;Log</title>\n<meta name="keywords" content="nlp, language-model, agent, steerability, prompting" />\n<meta name="description" content="Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent&rsquo;s brain, complemented by several key components:">\n<meta name="author" content="Lilian Weng">\n<link

### Preprocessor

In [3]:
preprocessor = GithubBlogpostPreprocessor(document_content)
cleaned_text = preprocessor.get_text()
display_md(cleaned_text[0:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In

### Splitter

In [4]:
text_splitter = SimpleCharacterTextSplitter(chunk_size=150, overlap_size=25)
text_splits = text_splitter.split_text(cleaned_text)
display_md(text_splits[1])

mistakes and refine them for future steps, thereby improving the quality of final results. Memory Short-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizing short-term memory of the model to learn. Long-term memory: This provides the agent with the capability to retain and recall (infinite) information over extended periods, often by leveraging an external vector store and fast retrieval. Tool use The agent learns to call external APIs for extra information that is missing from the model weights (often hard to change after pre-training), including current information, code execution capability, access to proprietary information sources and more. Fig. 1. Overview of a LLM-powered autonomous agent system. Component One: Planning# A complicated task usually involves many steps. An agent needs to know what they are and plan ahead. Task Decomposition# Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller

### Vector Store  

In [5]:
text_splits = text_splits
collection_name = "GithubBlogpostChunk"

vector_db = WeaviateVectorDb(text_splits=text_splits, collection_name=collection_name)
vector_db.count_entries()

44

### Retriever

In [6]:
question = "What is Task Decomposition?"
retrieved_context = vector_db.retrieve(question)
print(retrieved_context)

model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process. Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote. Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs. Another quite distinct approach, LLM+P (Liu et al

### Generator

In [7]:
generator = Gpt35Generator()
completion = generator.generate(question, retrieved_context)
display_md(completion)

Task decomposition is a technique used to break down complex tasks into smaller and simpler steps to enhance model performance. It involves transforming big tasks into multiple manageable tasks to facilitate interpretation of the model's thinking process. Different approaches like CoT and Tree of Thoughts can be used for task decomposition.